In [1]:
import warnings
warnings.filterwarnings('ignore')

from astropy.table import Table, hstack, vstack, join
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as ac
import astropy.units as au
from glob import glob
from spectral_cube import SpectralCube
import scipy 
from reproject import reproject_interp
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from tqdm.auto import tqdm 

from astropy.io import fits
import matplotlib as mpl
import pyregion
import aplpy
import math
import os
import pickle

In [2]:
"""Easy pickel"""

def save_pickle(a, filename): 
    with open(filename, 'wb') as handle:
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('[INFO] Saved to %s' %filename)
    return()

def load_pickle(filename):
    with open('%s'%filename, 'rb') as handle:
        b = pickle.load(handle)
    print('[INFO] Load %s' %filename)
    return(b)

In [3]:
hdus = load_pickle('../../analysis/catalogue/hdus_sample_withdendro.pickel')
asscat1_hdus = hdus['asscat1_hdus']
asscat2_hdus = hdus['asscat2_hdus']
asscat3_hdus = hdus['asscat3_hdus']
indexmap_trunk_hdus = hdus['indexmap_trunk_hdus']

[INFO] Load ../../analysis/catalogue/hdus_sample_withdendro.pickel


In [4]:
dir_asscat = '../../../data/cluster_catalogs/ngc628-vselect/'
asscat1_file = '%s/ws16pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws16pc-main.fits' %dir_asscat #HST association catalogue
asscat2_file = '%s/ws32pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws32pc-main.fits' %dir_asscat #HST association catalogue
asscat3_file = '%s/ws64pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws64pc-main.fits' %dir_asscat #HST association catalogue

asscat1_table = Table.read(asscat1_file)
asscat2_table = Table.read(asscat2_file)
asscat3_table = Table.read(asscat3_file)

def addunits(asscat_table):
    asscat_table['reg_dolflux_Age_MinChiSq'] = asscat_table['reg_dolflux_Age_MinChiSq'].quantity*au.Myr
    asscat_table['reg_dolflux_Mass_MinChiSq'] = asscat_table['reg_dolflux_Mass_MinChiSq'].quantity*au.Msun
    return(asscat_table)
    
asscat1_table = addunits(asscat1_table)
asscat2_table = addunits(asscat2_table)
asscat3_table = addunits(asscat3_table)

In [5]:
def get_regions(regions_file, hdu):
    
    regions = pyregion.open(regions_file).as_imagecoord(hdu.header)
    n = len(regions)
    
    ra = np.empty(n)*np.nan*au.deg
    dec = np.empty(n)*np.nan*au.deg
    width = np.empty(n)*np.nan*au.deg
    height = np.empty(n)*np.nan*au.deg
    position = ['']*n
    
    for i, region in enumerate(regions):

        ra[i] = float(region.params[0].text)*au.deg
        dec[i] = float(region.params[1].text)*au.deg
        width[i] = region.params[2].degree*au.deg
        height[i] = region.params[3].degree*au.deg
        
    position = SkyCoord(ra=ra, dec=dec, frame='icrs')

    return({'ra':ra, 'dec':dec, 'width':width, 'height':height, 'position': position})

regions_file = './../../analysis/catalogue/sample_v2.reg'
regions = get_regions(regions_file, hdus['hst07_hdus'][0])

In [6]:
def get_clusters(asscat_id, asscat_table, asscat_file):
    
    #add name to table to keep track
    asscat_table['asscat'] = asscat_file.split('/')[-1]

    #get ids if multiple are present within array
    asscat_table = asscat_table[np.searchsorted(asscat_table['reg_id'], asscat_id)]

    #get the minimum age clusters
    min_age = np.min(asscat_table['reg_dolflux_Age_MinChiSq'])
    asscat_table = asscat_table[asscat_table['reg_dolflux_Age_MinChiSq']==min_age]

    #if still multiple, only take the maximum mass
    max_mass = np.max(asscat_table['reg_dolflux_Mass_MinChiSq']) 
    asscat_table = asscat_table[asscat_table['reg_dolflux_Mass_MinChiSq']==max_mass]

    if len(asscat_table)>1:
        print('[INFO] STOP - please check, too many clusters!')
        
    return(asscat_table)

In [7]:
"""Load properties of the hst nebula catalouge"""
props_all = Table.read('../../analysis/dendro/props_all/props_all3sig_1.fits')
props_all

_idx,area_ellipse,area_exact,flux,major_sigma,minor_sigma,position_angle,mean_sigma,x_cen,y_cen,id,radius_trunk,radius_trunkclose,major_fwtm,minor_fwtm,mean_fwtm,mean_hwtm,min_npix,min_value_sig,min_delta_sig,radius_trunk_pc,radius_trunkclose_pc,major_fwtm_pc,minor_fwtm_pc,mean_fwtm_pc,mean_hwtm_pc,ra_cent,dec_cent,mean_sigma_pc,muscat_id,gal_name,cen_x,cen_y,cen_ra,cen_dec,deproj_dist,deproj_phi,region_area,region_circ_rad,HII_class,Environment,HA6562_FLUX,SII6716_FLUX,SII6730_FLUX,HA6562_FLUX_ERR,SII6716_FLUX_ERR,SII6730_FLUX_ERR,HA6562_SIGMA,HA6562_VEL,HA6562_FLUX_CORR,SII6716_FLUX_CORR,SII6730_FLUX_CORR,HA6562_FLUX_CORR_ERR,EBV,EBV_ERR,met_scal,met_scal_err,logq_D91,logq_D91_err,t_N2,t_S3,t_N2_err,t_S3_err,LOG_SIGMA_SFR_CHAB,LOG_SIGMA_STMASS_CHAB,dist_Mpc,dist_pc,kpc_per_pixel,pc_per_pixel,ha_lum,reff_pc,rmin_pc,Rgal_pc,Rgal_kpc,Rgal_r25,Rgal_reff,isHII,rfrac,resolved,reff_deconv,reff_deconv_pc,resolved_deconv,t_Pcal,ne,ne_sig,pth,rec_rate,rec_coeff,ne_Halphalowerlim,pth_Halphalowerlim,pde_classic_megatable,pde_classic,pde_smallscale,sigma_atom,sigma_mol,Lbol,Pdir,reff_Halphalowerlim,Pdir_Halphaupperlim,clustermass,mechanicalluminosity,windvelocity,pwind,pwind_upperlim,pth_min,pth_max,pdir_min,pdir_max,pwind_min,pwind_max,T_N2_REFIT
,arcsec2,arcsec2,erg / (cm2 s),arcsec,arcsec,deg,arcsec,pix,pix,,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,,,,pc,pc,pc,pc,pc,pc,deg,deg,pc,,,pix,pix,deg,deg,arcsec,rad,pix2,arcsec,,,erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),km / s,km / s,erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),mag,mag,,,,,K,K,K,K,solMass / (kpc2 yr),solMass / kpc2,Mpc,pc,kpc,pc,solLum,pc,pc,pc,kpc,,,,,,arcsec,pc,,K,1 / cm3,,K / cm3,1 / s,cm3 / s,1 / cm3,K / cm3,K / cm3,K / cm3,K / cm3,solMass / pc2,solMass / pc2,solLum,K / cm3,pc,K / cm3,,erg / s,,K / cm3,K / cm3,K / cm3,K / cm3,K / cm3,K / cm3,K / cm3,K / cm3,
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes32,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,bool,float64,float64,bool,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0,0.14232271564122573,0.4583653647999954,54072.78125,0.23613083991959577,0.13839601154891676,93.52926249469827,0.18077490547243494,34.806511411746826,29.148529442338603,26,0.38197150037676664,0.4270571201550184,1.013473564934905,0.5939956815679507,0.8037346232514277,0.4018673116257139,9,3,5,18.222188068705837,20.37302561020593,48.348386946904505,28.336933543088662,38.34266024499658,19.17133012249829,24.18324772349951,15.775644783434101,8.623979334510564,44.0,NGC0628,544.9798223906498,612.7191685718963,24.18324518770428,15.775690386879095,43.656311080219716,3.501822924541612,54.0,0.8291859587312053,1,6.0,56032.63075376848,8599.78467527863,6038.072900566596,176.09396604883048,92.07763692039325,89.56705278526734,18.54307968002811,-6.565568334008774,163347.80516730386,24396.887082130434,17087.40399271927,3923.0276365469213,0.46117509428297787,0.010262683527259037,8.637924152368807,0.0017556098100390569,7.424516784380719,0.022327581728051583,--,--,--,--,-1.742481368969421,8.22547651355552,9.84000015258789,9840000.15258789,0.009514815573393858,9.514815573393857,4916.365576694733,39.44775736687594,27.372351755271772,2082.6514808173292,2.0826514808173293,0.147206096301396,0.5336479327317929,True,1.802578171154794,False,--,--

In [8]:
n = len(regions['ra'])

for regionID in tqdm(range(n)): 

#     if regionID>10: 
#         continue
        
    asscat1_hdu = asscat1_hdus[regionID]
    asscat2_hdu = asscat2_hdus[regionID]
    asscat3_hdu = asscat3_hdus[regionID]
    indexmap_trunk_hdu = indexmap_trunk_hdus[regionID]
    
    mask_trunk = indexmap_trunk_hdu.data!=-1
    
    #regrid
    asscat1_data = reproject_interp(asscat1_hdu, indexmap_trunk_hdu.header, return_footprint=False, order='nearest-neighbor')
    asscat2_data = reproject_interp(asscat2_hdu, indexmap_trunk_hdu.header, return_footprint=False, order='nearest-neighbor')
    asscat3_data = reproject_interp(asscat3_hdu, indexmap_trunk_hdu.header, return_footprint=False, order='nearest-neighbor')
  
    #get clusters
    asscat1_data[~mask_trunk] = np.nan
    asscat2_data[~mask_trunk] = np.nan
    asscat3_data[~mask_trunk] = np.nan

    asscat1_id = np.unique(asscat1_data)[~np.isnan(np.unique(asscat1_data))]
    asscat2_id = np.unique(asscat2_data)[~np.isnan(np.unique(asscat2_data))]
    asscat3_id = np.unique(asscat3_data)[~np.isnan(np.unique(asscat3_data))]
  
    if len(asscat2_id)>0:
        asscat_table = get_clusters(asscat2_id, asscat2_table, asscat2_file)
        asscat_table.rename_column('reg_id', 'asscat2_id')
        
    elif len(asscat1_id)>0:
        asscat_table = get_clusters(asscat1_id, asscat1_table, asscat1_file)
        asscat_table.rename_column('reg_id', 'asscat1_id')
        
    elif len(asscat3_id)>0:
        asscat_table = get_clusters(asscat3_id, asscat3_table, asscat3_file)
        asscat_table.rename_column('reg_id', 'asscat3_id')
        
    else: 
        print('[INFO] No clusters for regionID: %i' %regionID)
        continue
    
    asscat_table['id'] = regionID
    
    if regionID == 0: 
        clust_all_final = asscat_table
    else: 
        clust_all_final = vstack([clust_all_final, asscat_table])

  0%|          | 0/215 [00:00<?, ?it/s]

[INFO] No clusters for regionID: 18
[INFO] No clusters for regionID: 21
[INFO] No clusters for regionID: 24
[INFO] No clusters for regionID: 25
[INFO] No clusters for regionID: 26


IndexError: list index out of range

In [9]:
props_all_final = join(props_all, clust_all_final, 'id')

In [10]:
save_pickle(props_all_final, '../../analysis/dendro/props_all/props_all_2.pickel') 
props_all_final.write('../../analysis/dendro/props_all/props_all_2.fits', overwrite=True)

[INFO] Saved to ../../analysis/dendro/props_all/props_all_2.pickel
